In [126]:
import torch
from torch.utils.data import DataLoader, Dataset
from torch import nn, optim
from transformers import AutoImageProcessor, AutoModelForImageClassification
from torchvision import transforms
from PIL import Image
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from pathlib import Path
from ultralytics import YOLO
from PIL import Image
import numpy as np
import cv2
import joblib
from tqdm import tqdm


In [127]:
object_det_model = YOLO("yolov8x.pt")

In [128]:
def rotate(img, angle):
    if angle == 0:
        return img
    h, w = img.size[1], img.size[0]
    m = cv2.getRotationMatrix2D((w / 2, h / 2), angle, 1)
    rad = np.radians(angle)
    nw = int(abs(np.sin(rad)) * h + abs(np.cos(rad)) * w)
    nh = int(abs(np.cos(rad)) * h + abs(np.sin(rad)) * w)
    m[0, 2] += (nw - w) / 2
    m[1, 2] += (nh - h) / 2
    return Image.fromarray(cv2.warpAffine(np.array(img), m, (nw, nh)))

In [129]:
def detect_best_class(model, img_path):
    img = Image.open(img_path).convert("RGB")
    angles = [0, 45, 90, 180]

    best = {"cls": None, "conf": 0}

    for a in angles:
        rimg = rotate(img, a)
        res = model(rimg)[0]

        if res.boxes:
            for b in res.boxes:
                conf = float(b.conf)
                cls = int(b.cls)

                if conf > best["conf"]:
                    best.update({"cls": cls, "conf": conf})

    if best["cls"] is None:
        return "None"

    return model.names[best["cls"]]

In [130]:
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=0.05):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        noise = torch.randn(tensor.size()) * self.std + self.mean
        return torch.clamp(tensor + noise, 0., 1.)

In [131]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),

    # 무작위 회전 (30, 45, 60도 중 하나)
    transforms.RandomChoice([
        transforms.RandomRotation(30),
        transforms.RandomRotation(45),
        transforms.RandomRotation(60),
        transforms.RandomRotation(75),
        transforms.RandomRotation(90),
        transforms.RandomRotation(120),
        transforms.RandomRotation(150),
        transforms.RandomRotation(180),
        transforms.RandomRotation(210),
        transforms.RandomRotation(240),
        transforms.RandomRotation(270),
        transforms.RandomRotation(300)
    ]),

    # 좌우 / 상하 반전
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),

    # 30% 확률로 흑백화
    transforms.RandomApply([transforms.Grayscale(num_output_channels=3)], p=0.3),

    transforms.ToTensor(),

    # 가우시안 노이즈 추가
    AddGaussianNoise(0., 0.05),
])

# 검증 및 테스트용은 변형 최소화 (노이즈, 회전 등 제외)
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

In [132]:
class CustomImageDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.data = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_path = f"{self.img_dir}/{row['ID']}"
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        label = torch.tensor(row["target"], dtype=torch.long) if "target" in self.data.columns else -1
        return image, label

In [133]:
processor = AutoImageProcessor.from_pretrained("Falconsai/nsfw_image_detection")
coarse_model = AutoModelForImageClassification.from_pretrained("Falconsai/nsfw_image_detection")

coarse_model.classifier = nn.Linear(coarse_model.classifier.in_features, 6)
coarse_model.load_state_dict(torch.load("/root/CV_/classfication_codes/coarse_model.pth"))
coarse_model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
coarse_model.to(device)

le_coarse = joblib.load("/root/CV_/classfication_codes/coarse_label_encoder.pkl")

In [134]:
def predict_coarse(img_path):
    img = Image.open(img_path).convert("RGB")
    img_tensor = test_transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        logits = coarse_model(img_tensor).logits
        pred = logits.argmax(1).item()

    return le_coarse.inverse_transform([pred])[0]

In [135]:
def split_by_coarse(df, img_dir):
    group_dict = {f"G{i}": [] for i in range(6)}

    for _, row in tqdm(df.iterrows(), total=len(df)):
        img_path = f"{img_dir}/{row['ID']}"
        coarse = predict_coarse(img_path)
        group_dict[coarse].append(row)

    return {k: pd.DataFrame(v) for k, v in group_dict.items()}

In [136]:
train_df = pd.read_csv("/root/CV_/datasets/data/train.csv")
img_dir  = "/root/CV_/datasets/data/train"

train_split = split_by_coarse(train_df, img_dir)

labelings = {}
fine_models = {}

100%|██████████| 1570/1570 [00:54<00:00, 28.96it/s]


In [137]:
def train_fine_model(group_name, df_group,epochs=20):
    print(f"Training Fine model for {group_name} ...")

    # 1) Label encoding
    le = LabelEncoder()
    df_group["target"] = le.fit_transform(df_group["target"])
    labelings[group_name] = le

    # 2) Dataset
    dataset = CustomImageDataset(df_group, img_dir, train_transform)
    loader = DataLoader(dataset, batch_size=16, shuffle=True)

    # 3) 모델 생성
    model = AutoModelForImageClassification.from_pretrained("Falconsai/nsfw_image_detection")
    num_classes = len(le.classes_)
    model.classifier = nn.Linear(model.classifier.in_features, num_classes)
    model.to(device)

    # 4) 학습
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=5e-5)

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images).logits
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"[{group_name}] Epoch {epoch+1}/{epochs} Loss={total_loss/len(loader):.4f}")

    fine_models[group_name] = model

In [138]:
train_fine_model("G2", train_split["G2"],epochs=40)
train_fine_model("G5", train_split["G5"],epochs=90)

Training Fine model for G2 ...
[G2] Epoch 1/40 Loss=0.5496
[G2] Epoch 2/40 Loss=0.1695
[G2] Epoch 3/40 Loss=0.0498
[G2] Epoch 4/40 Loss=0.0355
[G2] Epoch 5/40 Loss=0.0331
[G2] Epoch 6/40 Loss=0.0504
[G2] Epoch 7/40 Loss=0.0230
[G2] Epoch 8/40 Loss=0.0237
[G2] Epoch 9/40 Loss=0.0189
[G2] Epoch 10/40 Loss=0.0231
[G2] Epoch 11/40 Loss=0.0094
[G2] Epoch 12/40 Loss=0.0084
[G2] Epoch 13/40 Loss=0.0076
[G2] Epoch 14/40 Loss=0.0071
[G2] Epoch 15/40 Loss=0.0065
[G2] Epoch 16/40 Loss=0.0061
[G2] Epoch 17/40 Loss=0.0059
[G2] Epoch 18/40 Loss=0.0054
[G2] Epoch 19/40 Loss=0.0052
[G2] Epoch 20/40 Loss=0.0135
[G2] Epoch 21/40 Loss=0.0186
[G2] Epoch 22/40 Loss=0.0066
[G2] Epoch 23/40 Loss=0.0249
[G2] Epoch 24/40 Loss=0.0075
[G2] Epoch 25/40 Loss=0.0047
[G2] Epoch 26/40 Loss=0.0074
[G2] Epoch 27/40 Loss=0.0216
[G2] Epoch 28/40 Loss=0.0186
[G2] Epoch 29/40 Loss=0.0276
[G2] Epoch 30/40 Loss=0.0047
[G2] Epoch 31/40 Loss=0.0040
[G2] Epoch 32/40 Loss=0.0037
[G2] Epoch 33/40 Loss=0.0035
[G2] Epoch 34/40 Loss

In [139]:
def predict_final(img_path):
    coarse = predict_coarse(img_path)

    # ✅ YOLO로 차량 감지 → 강제 class 16
    pil_img = Image.open(img_path).convert("RGB")
    yolo_res = object_det_model(pil_img)[0]

    for box in yolo_res.boxes:
        if object_det_model.names[int(box.cls[0])] == "car":
            return 16

    # ✅ coarse가 G2 또는 G5 → fine 모델 사용
    if coarse in ["G2", "G5"]:
        model = fine_models[coarse]
        le = labelings[coarse]
        
        img = Image.open(img_path).convert("RGB")
        img_tensor = test_transform(img).unsqueeze(0).to(device)

        with torch.no_grad():
            logits = model(img_tensor).logits
            pred = logits.argmax(1).item()

        return le.inverse_transform([pred])[0]

    # ✅ coarse가 G0,G1,G3,G4 → coarse target을 그대로 반환
    group_original_targets = {
        "G0": [0],
        "G1": [2],
        "G2": [5,8],
        "G3": [9],
        "G4": [16],
        "G5": [1,3,4,6,7,10,11,12,13,14,15],
    }
    return group_original_targets[coarse][0]

In [ ]:
test_df = pd.read_csv("/root/CV_/datasets/data/sample_submission.csv")
test_img_dir = "/root/CV_/datasets/data/test"

preds = []
for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
    img_path = f"{test_img_dir}/{row['ID']}"
    pred = predict_final(img_path)
    preds.append(pred)

test_df["target"] = preds
test_df.to_csv("two_stage_classifier_output_v1.csv", index=False)

print("✅ 완료! 제출 파일 생성됨.")

  0%|          | 0/3140 [00:00<?, ?it/s]


0: 640x480 1 book, 2 clocks, 40.9ms
Speed: 2.3ms preprocess, 40.9ms inference, 6.0ms postprocess per image at shape (1, 3, 640, 480)


  0%|          | 1/3140 [00:00<38:30,  1.36it/s]


0: 480x640 (no detections), 13.6ms
Speed: 1.8ms preprocess, 13.6ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)


  0%|          | 2/3140 [00:00<19:48,  2.64it/s]

...

100%|█████████▉| 3137/3140 [02:31<00:00, 19.92it/s]


0: 416x640 1 clock, 12.5ms
Speed: 1.2ms preprocess, 12.5ms inference, 0.9ms postprocess per image at shape (1, 3, 416, 640)

0: 480x640 (no detections), 13.2ms
Speed: 1.4ms preprocess, 13.2ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 640x480 1 laptop, 13.2ms
Speed: 1.4ms preprocess, 13.2ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 480)


100%|██████████| 3140/3140 [02:31<00:00, 20.70it/s]

✅ 완료! 제출 파일 생성됨.


: 